In [1]:
import pandas as pd
import numpy as np

In [ ]:
raw_data = pd.read_excel('Data 3 tỉnh.xlsx')
raw_data.head()

In [33]:
# Remove unneeded columns
raw_data = raw_data.drop(columns=[
    'Tỷ lệ lao động qua đào tạo trên địa bàn tỉnh', 
    'Tỷ lệ lao động qua đào tạo có bằng cấp, chứng chỉ', 
    'Tỷ lệ hộ dân sử dụng nước sạch', 
    'Tỷ lệ hộ đô thị sử dụng nước sạch', 
    'Tỷ lệ hộ nông thôn sử dụng nước sạch', 
    'Tỷ lệ thu gom chất thải rắn sinh hoạt đô thị',
    'Tỷ lệ thu gom chất thải rắn sinh hoạt nông thôn',
    'Tỷ lệ che phủ rừng',
    ])


In [ ]:
raw_data

In [ ]:
raw_data.info()

In [ ]:
df_vinh_long = raw_data[raw_data['Tỉnh'] == 'Vĩnh Long']
df_vinh_long.head()

In [ ]:
df_ben_tre = raw_data[raw_data['Tỉnh'] == 'Bến Tre']
df_ben_tre.head()

In [ ]:
df_tra_vinh = raw_data[raw_data['Tỉnh'] == 'Trà Vinh']
df_tra_vinh.head()

In [39]:
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from prophet import Prophet
import logging
import warnings

# Suppress excessive logging and warnings
warnings.filterwarnings('ignore')
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

In [40]:
# Define forecast years
forecast_years = np.arange(2025, 2031).reshape(-1, 1)
forecast_df = pd.DataFrame({'Năm': forecast_years.flatten()})

In [ ]:
forecast_df

In [42]:
# Identify features to forecast (all numeric columns except 'Năm')
features_to_forecast = raw_data.select_dtypes(include=np.number).columns.tolist()
features_to_forecast.remove('Năm')

In [ ]:
features_to_forecast

In [44]:
all_forecasts = {}

In [45]:
def forecast_linear_regression(series: pd.Series):
    """Forecasts using Linear Regression based on Year."""
    df = series.reset_index()
    df.columns = ['Năm', 'Value']

    # Prepare data for sklearn
    X_train = df['Năm'].values.reshape(-1, 1)
    y_train = df['Value'].values

    # Handle potential NaN/inf values if any exist before fitting
    if not np.all(np.isfinite(y_train)):
         print(f"Warning: Non-finite values found in series {series.name}. Skipping linear regression.")
         return pd.Series(index=forecast_years.flatten(), dtype=float)


    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict
    predictions = model.predict(forecast_years)
    return pd.Series(predictions, index=forecast_years.flatten())

def forecast_10_percent_growth(series: pd.Series):
    """Forecasts assuming 10% compound annual growth from the last value."""
    last_year = series.index.max()
    last_value = series.loc[last_year]

    # Handle potential NaN/inf values
    if not np.isfinite(last_value):
        print(f"Warning: Last value for {series.name} is non-finite. Cannot apply 10% growth.")
        return pd.Series(index=forecast_years.flatten(), dtype=float)


    forecast_values = []
    for year in forecast_years.flatten():
        growth_factor = 1.1 ** (year - last_year)
        forecast_values.append(last_value * growth_factor)
    return pd.Series(forecast_values, index=forecast_years.flatten())

def forecast_arima(series: pd.Series,
                         forecast_index: pd.Index,
                         min_points: int = 6) -> pd.Series:
    """
    ARIMA forecast for very short, non‑seasonal series (≥ 6 points).

    Parameters
    ----------
    series : pd.Series
        The historical data (numeric), indexed by a DatetimeIndex or PeriodIndex.
    forecast_index : pd.Index
        Index for the horizon to predict (e.g., years to come).
    min_points : int, default 6
        Hard floor below which we refuse to fit any ARIMA model.

    Returns
    -------
    pd.Series
        Point forecasts, indexed by `forecast_index`.
    """

    y = series.dropna()

    # --- basic validity checks ------------------------------------------------
    if len(y) < min_points or not np.all(np.isfinite(y)):
        # Not enough data or non‑finite values → naïve forecast
        print(f"[ARIMA] {series.name}: fallback to last‑value forecast "
              f"(n = {len(y)}).")
        return pd.Series(y.iloc[-1] if len(y) else np.nan,
                         index=forecast_index, dtype=float)

    if y.std() < 1e-9:
        # Constant series → persistence
        return pd.Series(y.iloc[-1], index=forecast_index, dtype=float)

    # --- very restricted search to avoid over‑fitting -------------------------
    try:
        arima = auto_arima(
            y,
            start_p=0, start_q=0,
            max_p=1,  max_q=1,     # at most ARIMA(1,*,1)
            max_d=1,               # 0 or 1 diff only
            seasonal=False,
            information_criterion="aic",
            test='adf',            # decide d via ADF
            stepwise=False,        # brute‑force (tiny grid)
            error_action='ignore',
            suppress_warnings=True
        )
        print(f"[ARIMA] {series.name}: selected order {arima.order}")

        preds = arima.predict(n_periods=len(forecast_index))
        return pd.Series(preds, index=forecast_index)

    except Exception as exc:
        print(f"[ARIMA] {series.name}: model failed ({exc}). "
              "Reverting to naïve forecast.")
        return pd.Series(y.iloc[-1], index=forecast_index, dtype=float)


def forecast_prophet(series: pd.Series):
    """Forecasts using Facebook Prophet."""
    df = series.reset_index()
    df.columns = ['Năm', 'y']
    df['ds'] = pd.to_datetime(df['Năm'].astype(str) + '-12-31') # Use year-end for annual data

     # Handle potential NaN/inf values
    if not np.all(np.isfinite(df['y'].values)):
         print(f"Warning: Non-finite values found in series {series.name}. Skipping Prophet.")
         return pd.Series(index=forecast_years.flatten(), dtype=float)


    try:
        model = Prophet(growth='linear', yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
        model.fit(df[['ds', 'y']])

        # Create future dataframe
        future_dates = model.make_future_dataframe(periods=len(forecast_years), freq='A-DEC') # Annual frequency ending Dec 31st
        # Adjust future dates to match target years if needed
        future_dates['year'] = future_dates['ds'].dt.year
        future_dates = future_dates[future_dates['year'].isin(forecast_years.flatten())]


        # Predict
        forecast_result = model.predict(future_dates)
        predictions = forecast_result[['ds', 'yhat']].set_index('ds')['yhat']
        predictions.index = predictions.index.year # Use year as index
        # Ensure all forecast years are present, filling missing ones potentially with NaN
        predictions = predictions.reindex(forecast_years.flatten())
        return predictions
    except Exception as e:
        print(f"Error fitting Prophet for {series.name}: {e}")
        return pd.Series(index=forecast_years.flatten(), dtype=float)


In [46]:
province_dfs = {
    "Vĩnh Long": df_vinh_long,
    "Bến Tre": df_ben_tre,
    "Trà Vinh": df_tra_vinh
}

In [47]:
columns_for_10_percent = [
    "GRDP theo giá so sánh (tỷ dồng)",
    "GRDP theo giá hiện hành",
    "Thu ngân sách"
]

In [ ]:
for province_name, df_province in province_dfs.items():
    print(f"--- Forecasting for {province_name} ---")
    province_forecasts = {}
    # Ensure Năm is index and sorted, keep it as integer for now
    df_province_indexed = df_province.set_index('Năm').sort_index()

    for feature in features_to_forecast:
        print(f"   Forecasting {feature}...")
        series_int_index = df_province_indexed[feature].astype(float) # Series with integer index

        # Check if series is constant or mostly zero/NaN based on original data
        if series_int_index.nunique() <= 1 or not np.any(np.isfinite(series_int_index)):
             print(f"      Skipping {feature} for {province_name} due to constant value or all non-finite values.")
             feature_forecasts = pd.DataFrame(index=forecast_years.flatten())
             feature_forecasts['Linear Regression'] = np.nan
             feature_forecasts['10% Growth'] = np.nan
             feature_forecasts['ARIMA'] = np.nan
             feature_forecasts['Prophet'] = np.nan
        else:
             # --- Handle "Dân số" separately ---
             if feature == "Dân số":
                 print(f"      Handling '{feature}' feature: Only calculating Linear Regression.")
                 feature_forecasts = pd.DataFrame(index=forecast_years.flatten())
                 # Apply clipping here too for consistency, though less likely needed for Linear Regression on Population
                 lin_reg_preds = forecast_linear_regression(series_int_index)
                 # Clipping is likely irrelevant for Dân số, but harmless to include the check pattern
                 # if feature == "Tỷ lệ hộ nghèo của tỉnh":
                 #    print(f"      Applying lower bound 0 for Linear Regression on {feature}")
                 #    lin_reg_preds = lin_reg_preds.clip(lower=0)
                 feature_forecasts['Linear Regression'] = lin_reg_preds

                 feature_forecasts['10% Growth'] = np.nan # Explicitly NaN
                 feature_forecasts['ARIMA'] = np.nan      # Explicitly NaN
                 feature_forecasts['Prophet'] = np.nan     # Explicitly NaN
             else:
                 # --- Handle all other features ---
                 feature_forecasts = pd.DataFrame(index=forecast_years.flatten())

                 # Calculate Linear Regression and apply clipping if needed
                 lin_reg_preds = forecast_linear_regression(series_int_index)
                 if feature == "Tỷ lệ hộ nghèo của tỉnh":
                     print(f"      Applying lower bound 0 for Linear Regression on {feature}")
                     lin_reg_preds = lin_reg_preds.clip(lower=0)
                 feature_forecasts['Linear Regression'] = lin_reg_preds

                 # Calculate Prophet and apply clipping if needed
                 prophet_preds = forecast_prophet(series_int_index)
                 if feature == "Tỷ lệ hộ nghèo của tỉnh":
                     print(f"      Applying lower bound 0 for Prophet on {feature}")
                     prophet_preds = prophet_preds.clip(lower=0)
                 feature_forecasts['Prophet'] = prophet_preds

                 # Calculate 10% Growth conditionally and apply clipping if needed
                 if feature in columns_for_10_percent:
                     growth_preds = forecast_10_percent_growth(series_int_index)
                     if feature == "Tỷ lệ hộ nghèo của tỉnh":
                         print(f"      Applying lower bound 0 for 10% Growth on {feature}")
                         growth_preds = growth_preds.clip(lower=0)
                     feature_forecasts['10% Growth'] = growth_preds
                 else:
                     feature_forecasts['10% Growth'] = np.nan # Assign NaN if not included

                 # Prepare series with PeriodIndex for ARIMA and apply clipping if needed
                 try:
                     series_period_index = series_int_index.copy()
                     series_period_index.index = pd.PeriodIndex(series_period_index.index, freq='A-DEC', name='Năm')
                     arima_preds = forecast_arima(series_period_index)
                     if feature == "Tỷ lệ hộ nghèo của tỉnh":
                         print(f"      Applying lower bound 0 for ARIMA on {feature}")
                         arima_preds = arima_preds.clip(lower=0)
                     feature_forecasts['ARIMA'] = arima_preds
                 except Exception as e:
                     print(f"      Error creating PeriodIndex or running ARIMA for {feature} in {province_name}: {e}")
                     feature_forecasts['ARIMA'] = pd.Series(index=forecast_years.flatten(), dtype=float)

        province_forecasts[feature] = feature_forecasts

    # Store province results
    all_forecasts[province_name] = pd.concat(province_forecasts, axis=1)

print("\n--- Province-level forecasting complete ---")


In [ ]:
print("\n--- Merging Provinces and Forecasting ---")
# Ensure 'Năm' is a column for merging, then set back to index for summing
df_vinh_long_yr = df_vinh_long.reset_index()
df_ben_tre_yr = df_ben_tre.reset_index()
df_tra_vinh_yr = df_tra_vinh.reset_index()

In [ ]:
df_vinh_long_yr

In [51]:
# Merge based on year, sum numeric columns
df_merged = pd.merge(df_vinh_long_yr, df_ben_tre_yr, on=['Năm', 'Tỉnh'], how='outer', suffixes=('_VL', '_BT'))
df_merged = pd.merge(df_merged, df_tra_vinh_yr, on=['Năm', 'Tỉnh'], how='outer', suffixes=('', '_TV'))

In [ ]:
df_merged.head()

In [ ]:
# Group by year and sum all numeric columns (which should be the features)
df_merged_sum = raw_data.groupby('Năm')[features_to_forecast].sum().sort_index()
df_merged_sum.head()

In [54]:
merged_forecasts_dict = {}

In [ ]:
for feature in features_to_forecast:
    print(f"   Forecasting Merged {feature}...")
    # df_merged_sum already has 'Năm' (integer) as index
    series_int_index = df_merged_sum[feature].astype(float)

    # Check if series is constant or mostly zero/NaN
    if series_int_index.nunique() <= 1 or not np.any(np.isfinite(series_int_index)):
        print(f"      Skipping Merged {feature} due to constant value or all non-finite values.")
        feature_forecasts = pd.DataFrame(index=forecast_years.flatten())
        feature_forecasts['Linear Regression'] = np.nan
        feature_forecasts['10% Growth'] = np.nan
        feature_forecasts['ARIMA'] = np.nan
        feature_forecasts['Prophet'] = np.nan
    else:
        # --- Handle "Dân số" separately ---
        if feature == "Dân số":
            print(f"      Handling Merged '{feature}' feature: Only calculating Linear Regression.")
            feature_forecasts = pd.DataFrame(index=forecast_years.flatten())
            # Apply clipping here too for consistency
            lin_reg_preds = forecast_linear_regression(series_int_index)
            # Clipping is likely irrelevant for Dân số
            # if feature == "Tỷ lệ hộ nghèo của tỉnh":
            #     print(f"      Applying lower bound 0 for Linear Regression on Merged {feature}")
            #     lin_reg_preds = lin_reg_preds.clip(lower=0)
            feature_forecasts['Linear Regression'] = lin_reg_preds
            feature_forecasts['10% Growth'] = np.nan
            feature_forecasts['ARIMA'] = np.nan
            feature_forecasts['Prophet'] = np.nan
        else:
            # --- Handle all other features ---
            feature_forecasts = pd.DataFrame(index=forecast_years.flatten())

            # Calculate Linear Regression and apply clipping if needed
            lin_reg_preds = forecast_linear_regression(series_int_index)
            if feature == "Tỷ lệ hộ nghèo của tỉnh":
                print(f"      Applying lower bound 0 for Linear Regression on Merged {feature}")
                lin_reg_preds = lin_reg_preds.clip(lower=0)
            feature_forecasts['Linear Regression'] = lin_reg_preds

            # Calculate Prophet and apply clipping if needed
            prophet_preds = forecast_prophet(series_int_index)
            if feature == "Tỷ lệ hộ nghèo của tỉnh":
                print(f"      Applying lower bound 0 for Prophet on Merged {feature}")
                prophet_preds = prophet_preds.clip(lower=0)
            feature_forecasts['Prophet'] = prophet_preds

            # Calculate 10% Growth conditionally and apply clipping if needed
            if feature in columns_for_10_percent:
                growth_preds = forecast_10_percent_growth(series_int_index)
                if feature == "Tỷ lệ hộ nghèo của tỉnh":
                     print(f"      Applying lower bound 0 for 10% Growth on Merged {feature}")
                     growth_preds = growth_preds.clip(lower=0)
                feature_forecasts['10% Growth'] = growth_preds
            else:
                feature_forecasts['10% Growth'] = np.nan

            # Prepare series with PeriodIndex for ARIMA and apply clipping if needed
            try:
                series_period_index = series_int_index.copy()
                series_period_index.index = pd.PeriodIndex(series_period_index.index, freq='A-DEC', name='Năm')
                arima_preds = forecast_arima(series_period_index)
                if feature == "Tỷ lệ hộ nghèo của tỉnh":
                    print(f"      Applying lower bound 0 for ARIMA on Merged {feature}")
                    arima_preds = arima_preds.clip(lower=0)
                feature_forecasts['ARIMA'] = arima_preds
            except Exception as e:
                print(f"      Error creating PeriodIndex or running ARIMA for Merged {feature}: {e}")
                feature_forecasts['ARIMA'] = pd.Series(index=forecast_years.flatten(), dtype=float)

    merged_forecasts_dict[feature] = feature_forecasts

# Store merged results
all_forecasts["Merged"] = pd.concat(merged_forecasts_dict, axis=1)

print("\n--- Merged province forecasting complete ---")

In [ ]:
print("\n--- Forecast Results ---")

for name, forecasts in all_forecasts.items():
    print(f"\n--- {name} Forecasts (2025-2030) ---")
    # Show results for a key indicator, e.g., GRDP theo giá so sánh (tỷ dồng)
    grdp_col_name = 'GRDP theo giá so sánh (tỷ dồng)'
    if grdp_col_name in forecasts.columns.get_level_values(0):
         print(f"\nForecasts for: {grdp_col_name}")
         print(forecasts[grdp_col_name])
    else:
         print(f"\n{grdp_col_name} not found in forecasts for {name}.")


    # Optionally display all forecasts (can be very large)
    # print("\nFull Forecast Table:")
    # print(forecasts)

    # Example: Accessing forecast for a specific feature and model
    # try:
    #     print("\nExample: Vĩnh Long - GRDP theo giá hiện hành - Prophet Forecast:")
    #     print(all_forecasts['Vĩnh Long'][('GRDP theo giá hiện hành', 'Prophet')])
    # except KeyError as e:
    #     print(f"Could not access example forecast: {e}")

In [57]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker # For formatting axes if needed
import os # Import os module for directory operations
import re # Import re for cleaning filenames

In [ ]:
print("\n--- Generating and Saving Forecast Visualizations with Table (No Point Labels, Renamed Merged) ---")

# Define the output directory
output_dir = 'output_plots_with_table' # Keep same directory
os.makedirs(output_dir, exist_ok=True)

# Ensure historical dataframes have 'Năm' as index for plotting
df_vinh_long_hist = df_vinh_long.set_index('Năm')
df_ben_tre_hist = df_ben_tre.set_index('Năm')
df_tra_vinh_hist = df_tra_vinh.set_index('Năm')
# df_merged_sum is already indexed by 'Năm'

historical_data_map = {
    "Vĩnh Long": df_vinh_long_hist,
    "Bến Tre": df_ben_tre_hist,
    "Trà Vinh": df_tra_vinh_hist,
    "Merged": df_merged_sum # Keep key as "Merged" for lookup
}

# Define colors for different models
colors = {
    'Historical': 'black',
    'Linear Regression': 'blue',
    '10% Growth': 'red',
    'ARIMA': 'green',
    'Prophet': 'purple'
}
# Define models to include in table (can be adjusted)
models_for_table = ['Historical','Linear Regression', '10% Growth', 'ARIMA', 'Prophet']

In [ ]:
for name, forecasts in all_forecasts.items():
    # Use the display name for titles/filenames
    display_name = "Sát nhập 3 tỉnh" if name == "Merged" else name
    print(f"\nProcessing plots for: {display_name}")
    historical_df = historical_data_map[name] # Use original key for lookup

    for feature in features_to_forecast:
        # Check if the feature exists in both historical and forecast data
        if feature not in historical_df.columns:
             print(f"  Skipping plot for '{feature}' in '{display_name}' (historical data missing).")
             continue
        # Also check forecast data existence
        forecast_feature_exists = False
        if feature in forecasts.columns.get_level_values(0):
             forecast_feature_exists = True


        # Prepare data for the table
        hist_series = historical_df[feature].astype(float)
        table_data_list = [hist_series.rename('Historical')]

        if forecast_feature_exists:
            feature_forecasts = forecasts[feature]
            for model_name in models_for_table:
                 # Skip 'Historical' as it's already added
                 if model_name == 'Historical':
                     continue
                 if model_name in feature_forecasts.columns:
                     table_data_list.append(feature_forecasts[model_name].astype(float).rename(model_name))
                 else:
                     # Add an empty series if model forecast doesn't exist
                     table_data_list.append(pd.Series(name=model_name, dtype=float))

            # Combine all series into a DataFrame, filling NaNs for non-overlapping years
            # Only include years present in historical or any forecast
            combined_index = hist_series.index.union(feature_forecasts.index)
            table_df = pd.concat(table_data_list, axis=1).reindex(combined_index).sort_index()
            table_df.index.name = 'Năm'
            table_df = table_df.applymap(lambda x: f'{x:,.1f}' if pd.notna(x) else '') # Format numbers
            table_df = table_df.reset_index() # Make 'Năm' a column for the table function
        else:
            # If no forecast data, just format historical for the table
            print(f"  Feature '{feature}' not found in forecast data for '{display_name}'. Table will only show historical.")
            table_df = hist_series.reset_index()
            table_df.columns = ['Năm', 'Historical']
            table_df['Historical'] = table_df['Historical'].apply(lambda x: f'{x:,.1f}' if pd.notna(x) else '')


        # Start plotting
        # Adjust figsize: slightly wider, taller to give table more room
        fig, ax = plt.subplots(figsize=(15, 12)) # Use fig, ax for easier table positioning

        # --- Plotting Lines and Points (No text labels) ---
        hist_series_plot = hist_series.dropna()
        if not hist_series_plot.empty:
            ax.plot(hist_series_plot.index, hist_series_plot, label='Historical', marker='o', color=colors['Historical'], markersize=4)
            # Removed: ax.text() for historical points

        if forecast_feature_exists:
             feature_forecasts_plot = forecasts[feature]
             for model_name in feature_forecasts_plot.columns:
                 forecast_series_plot = feature_forecasts_plot[model_name].astype(float).dropna()
                 if not forecast_series_plot.empty:
                     color = colors.get(model_name, 'gray')
                     ax.plot(forecast_series_plot.index, forecast_series_plot, label=model_name, marker='x', linestyle='--', color=color, markersize=4)
                     # Removed: ax.text() for forecast points
        # --- End Plotting Lines and Points ---


        ax.set_title(f'{display_name} - {feature} Forecast (2025-2030)')
        ax.set_xlabel('Năm (Year)')
        ax.set_ylabel('Value')
        # Move legend to upper right to avoid potential overlap with steep lines
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.6)
        # Removed: ax.margins(y=0.1)

        # Adjust x-axis ticks
        all_years_indices = table_df['Năm'].tolist() # Get years from table data
        all_years = sorted(list(set(all_years_indices)))
        plt.xticks(all_years, rotation=45) # Set ticks explicitly to all years in the table
        # ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=len(all_years), integer=True)) # Alternative ticker approach
        ax.tick_params(axis='x', labelsize=8) # Smaller x-tick labels
        ax.tick_params(axis='y', labelsize=9)


        # --- Add Table ---
        if table_df is not None:
             # Adjust plot area: make bottom margin larger, top margin slightly smaller
             plt.subplots_adjust(left=0.2, bottom=0.35, right=0.93, top=0.85)

             # Filter table_df to only include relevant columns for display if needed
             display_cols = ['Năm'] + [col for col in models_for_table if col in table_df.columns]
             table_display_df = table_df[display_cols]


             # Add the table
             the_table = ax.table(cellText=table_display_df.values,
                                  colLabels=table_display_df.columns,
                                  loc='bottom', # Position table below the axes
                                  cellLoc='center',
                                  edges='horizontal',
                                  # Adjust bbox: [left, bottom, width, height]
                                  # bottom value (-0.6) pushes it further down, height (0.4) makes it taller
                                  bbox=[0.0, -0.7, 1.0, 0.6]
                                )

             the_table.auto_set_font_size(False)
             the_table.set_fontsize(8)
             the_table.scale(1, 1.1) # Adjust scale slightly
        else:
             plt.tight_layout() # Use tight layout if no table


        # Create a clean filename using display_name
        clean_feature_name = re.sub(r'[\\/*?:"<>|()]', '', feature)
        clean_feature_name = clean_feature_name.replace(' ', '_').replace('.', '')
        # Use display_name (which might contain spaces) cleaned for filename
        clean_name = display_name.replace(' ', '_')
        filename = f"{clean_name}_{clean_feature_name}_with_table.png"
        filepath = os.path.join(output_dir, filename)

        # Save the figure
        try:
             plt.savefig(filepath, dpi=150, bbox_inches='tight') # Use bbox_inches='tight' to help fit table
             print(f"  Saved plot with table: {filepath}")
        except Exception as e:
             print(f"  Error saving plot {filepath}: {e}")
        plt.show()
        plt.close(fig) # Close the figure explicitly

print(f"\n--- All visualizations with tables saved to '{output_dir}' directory ---")

In [ ]:
print("\n--- Exporting Forecast Results to Excel ---")

# Define the output Excel filename
excel_filename = 'forecast_results.xlsx'

try:
    # Create an Excel writer object
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        for name, forecasts_df in all_forecasts.items():
            # Clean the name for the sheet title (Excel sheet names have limitations)
            # Replace invalid chars, limit length (optional but good practice)
            clean_sheet_name = re.sub(r'[\\/*?:"<>|\[\]]', '', name) # Remove invalid Excel sheet chars
            clean_sheet_name = clean_sheet_name[:30] # Limit sheet name length (Excel limit is 31)

            display_name = "Sát nhập 3 tỉnh" if name == "Merged" else clean_sheet_name

            print(f"  Writing sheet: {display_name}")

            # Prepare DataFrame for export: Reset index to include 'Năm' as a column
            df_to_export = forecasts_df.copy()
            # Flatten MultiIndex columns for easier reading in Excel (optional but helpful)
            # Format: 'Feature - Model'
            df_to_export.columns = [' - '.join(col).strip() for col in df_to_export.columns.values]
            df_to_export = df_to_export.reset_index().rename(columns={'index': 'Năm'})


            # Write the dataframe to a specific sheet
            df_to_export.to_excel(writer, sheet_name=display_name, index=False)

    print(f"\n--- Forecast results successfully exported to '{excel_filename}' ---")

except Exception as e:
    print(f"\nError exporting forecasts to Excel: {e}")
    # You might need to install the engine: pip install openpyxl